In [1]:
import re
import json

from bs4 import BeautifulSoup
from bs4.element import NavigableString, Tag

In [2]:
with open("/home/kushal/Projects/llmlegalassistant/INLP-WS23/datasets/articles/htmls/31961D1009.html", "rb") as htmlfile:
    html_doc = htmlfile.read()

In [3]:
html_soup = BeautifulSoup(html_doc, "html.parser")

In [4]:
def extract_annex_segments(html_soup):
    annex_segments = []
    for soup in html_soup:
        if soup.name == "html":
            for html in soup:
                if html.name == "body":
                    for tag in html:
                        if "span" == tag.name:
                            continue
                        if "ANNEX" in tag.get_text():
                            annex_segments.append(tag)

    return annex_segments

In [5]:
def get_tag_definitions(segments):
    definitions = []
    for segment in segments:
        for child in segment:
            # skip if it is a navigable string (e.g., <p></p> for newline)
            if isinstance(child, NavigableString):
                continue
            # only process if the child is a tag
            if isinstance(child, Tag):
                if child.has_attr("class"):
                    if not ("doc-ti" in child["class"] and "ANNEX" in child.get_text())  and "ti-section-1" in child["class"]:
                        definitions.append({
                            "id": re.sub("\n *", "", child.string),
                            "title": re.sub("\n *", "", child.findNext().string)
                        })
    return definitions

In [6]:
annex_segments = extract_annex_segments(html_soup)
annex_definitions = get_tag_definitions(annex_segments)

annexes = []
for i, annex_definition in enumerate(annex_definitions):
    for annex_segment in annex_segments:
        content = ""
        annex_definition_body_flag = False
        for j, child in enumerate(annex_segment):
                if isinstance(child, NavigableString):
                    continue
                if isinstance(child, Tag):
                    # if child is empty
                    if child.string is None:
                        continue

                    child_string = re.sub("\n *", "", child.string)

                    if j+2 == len(annex_segment):
                        print("Last")
                        annex_definition_body_flag = False
                        annexes.append({
                            "id": annex_definition["id"],
                            "title": content
                        })
                        content = ""
                        break
                    elif i+1 < len(annex_definitions) and annex_definitions[i+1]["id"] == child_string:
                        annex_definition_body_flag = False
                        annexes.append({
                            "id": annex_definition["id"],
                            "title": content
                        })
                        content = ""
                        break

                    if annex_definition["id"] == child_string:
                        annex_definition_body_flag = True
                        content = child_string + " "
                    elif annex_definition_body_flag:
                        content += child_string

with open("./test-data/chunks/annex/annex.jsonl", "w") as jsonfile:
    json.dump(annexes, jsonfile)

Last


FileNotFoundError: [Errno 2] No such file or directory: './test-data/annex/annex.jsonl'

TITLE I Objects — Name — Seat — Duration
TITLE II Capital — Shares
TITLE III Administration of the Company
TITLE IV Auditors
TITLE V General meetings
SECTION I Provisions applicable to both ordinary and extraordinary general meetings
SECTION II Ordinary general meetings
SECTION III Extraordinary general meetings
TITLE VI Schedule of assets and liabilities — Distribution of Profits
TITLE VII Dissolution — Winding Up
TITLE VIII Disputes
TITLE IX Transitional provisions

['DEFAULT_INTERESTING_STRING_TYPES', 'EMPTY_ELEMENT_EVENT', 'END_ELEMENT_EVENT', 'START_ELEMENT_EVENT', 'STRING_ELEMENT_EVENT', '__bool__', '__call__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__unicode__', '__weakref__', '_all_strings', '_clone', '_event_stream', '_find_all', '_find_one', '_format_tag', '_indent_string', '_is_xml', '_lastRecursiveChild', '_last_descendant', '_namespaces', '_should_pretty_print', 'append', 'attrs', 'can_be_empty_element', 'cdata_list_attributes', 'childGenerator', 'children', 'clear', 'contents', 'css', 'decode', 'decode_contents', 'decompose', 'decomposed', 'default', 'descendants', 'encode', 'encode_contents', 'extend', 'extract', 'fetchNextSiblings', 'fetchParents', 'fetchPrevious', 'fetchPreviousSiblings', 'find', 'findAll', 'findAllNext', 'findAllPrevious', 'findChild', 'findChildren', 'findNext', 'findNextSibling', 'findNextSiblings', 'findParent', 'findParents', 'findPrevious', 'findPreviousSibling', 'findPreviousSiblings', 'find_all', 'find_all_next', 'find_all_previous', 'find_next', 'find_next_sibling', 'find_next_siblings', 'find_parent', 'find_parents', 'find_previous', 'find_previous_sibling', 'find_previous_siblings', 'format_string', 'formatter_for_name', 'get', 'getText', 'get_attribute_list', 'get_text', 'has_attr', 'has_key', 'hidden', 'index', 'insert', 'insert_after', 'insert_before', 'interesting_string_types', 'isSelfClosing', 'is_empty_element', 'known_xml', 'name', 'namespace', 'next', 'nextGenerator', 'nextSibling', 'nextSiblingGenerator', 'next_element', 'next_elements', 'next_sibling', 'next_siblings', 'parent', 'parentGenerator', 'parents', 'parserClass', 'parser_class', 'prefix', 'preserve_whitespace_tags', 'prettify', 'previous', 'previousGenerator', 'previousSibling', 'previousSiblingGenerator', 'previous_element', 'previous_elements', 'previous_sibling', 'previous_siblings', 'recursiveChildGenerator', 'renderContents', 'replaceWith', 'replaceWithChildren', 'replace_with', 'replace_with_children', 'select', 'select_one', 'self_and_descendants', 'setup', 'smooth', 'sourceline', 'sourcepos', 'string', 'strings', 'stripped_strings', 'text', 'unwrap', 'wrap']
